In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC

import pickle
import numpy as np

import sys
sys.path.append('..')

In [37]:
def get_data():
    with open('../data/all_features.pkl', 'rb') as f:
        data = pickle.load(f)

    x = np.array(data[0])
    y = np.array(data[1])
    
    x = MinMaxScaler().fit_transform(x)

    return x, y

def test_accuracy(model):
    x, y = get_data()
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2)

    model.fit(x_train, y_train)
    results = model.predict(x_test)

    print(classification_report(y_test, results))

x, y = get_data()

## Logistic Regression

In [ ]:
model = LogisticRegression(solver='newton-cg', random_state=42, n_jobs=-1)
test_accuracy(model)

In [ ]:
model = GridSearchCV(LogisticRegression(solver='newton-cg', random_state=42, n_jobs=-1), {
    
}, cv=3, return_train_score=False)

model.fit()

## SVM

In [ ]:
model = SVC(kernel='linear', probability=True, random_state=42)
test_accuracy(model)